In [51]:
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense, Reshape, Flatten
from time import gmtime, strftime

# Data loading and processing

In [52]:
x_train_path = "data/trainData.npy"
y_train_path = "data/trainTarget.npy"

In [53]:
def loadnpy(path):
    data = np.load(path)
    return data

In [54]:
x_train = loadnpy(x_train_path)
y_train = loadnpy(y_train_path)

# (7, 13)
#print(x_train[0].shape)
# (4, 13)
#print(y_train[0].shape)

# Build Model

In [73]:
def bulidModel(x_shape):
    model = Sequential()
    model.add(Dense(units=256, input_shape=(7, 13), activation='relu'))
    model.add(Dense(units=64,activation='relu'))
    model.add(Dense(units=32,activation='relu'))
    model.add(Dense(units=16,activation='relu'))
    model.add(Dense(units=52,activation='relu'))
    model.add(Flatten())
    #model.add(Reshape((4, 13)))
    model.summary()
    
    return model


In [74]:
print("Build Model")

model = bulidModel(x_train.shape)
model.compile(loss='mse', optimizer='rmsprop')#adam

print("Model Training")
model_history = model.fit(x_train , y_train , batch_size=64 , epochs=100 , verbose=1, validation_split=0.2, shuffle=True)

time = strftime("%Y%m%d_%H%M%S", gmtime())

model.save('model+'+time+'.h5')

Build Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_134 (Dense)            (None, 7, 256)            3584      
_________________________________________________________________
dense_135 (Dense)            (None, 7, 64)             16448     
_________________________________________________________________
dense_136 (Dense)            (None, 7, 32)             2080      
_________________________________________________________________
dense_137 (Dense)            (None, 7, 16)             528       
_________________________________________________________________
dense_138 (Dense)            (None, 7, 52)             884       
_________________________________________________________________
flatten_5 (Flatten)          (None, 364)               0         
Total params: 23,524
Trainable params: 23,524
Non-trainable params: 0
____________________________________________________________

ValueError: Error when checking target: expected flatten_5 to have 2 dimensions, but got array with shape (2000000, 4, 13)